In [1]:
%load_ext autoreload
%autoreload 2

# Initialize DB and Tables

In [2]:
from package.databases.initialize import init_db_and_tables

init_db_and_tables()

In [3]:
from sqlmodel import Session, select, Text
from package.databases.session import get_session, Depends
from package.databases.models import User, Project

# CRUD

## Create

In [4]:
def create_user(user:User, session: Session = Depends(get_session)):
    session.add(user)
    session.commit()
    session.refresh(user)
    session.close()
    return user

bank = User(username="bank", password="test", email="bank.bank@bank.com")
bonk = User(username="bonk", password="test", email="bonk.bonk@bank.com")
users = [bank, bonk]
for user in users:
    print(create_user(user=user))

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "ix_user_username"
DETAIL:  Key (username)=(bank) already exists.

[SQL: INSERT INTO "user" (username, password, email, created_dt, updated_dt) VALUES (%(username)s, %(password)s, %(email)s, %(created_dt)s, %(updated_dt)s) RETURNING "user".user_id]
[parameters: {'username': 'bank', 'password': 'test', 'email': 'bank.bank@bank.com', 'created_dt': datetime.datetime(2025, 6, 27, 17, 29, 12, 811405, tzinfo=datetime.timezone.utc), 'updated_dt': datetime.datetime(2025, 6, 27, 17, 29, 12, 811405, tzinfo=datetime.timezone.utc)}]
(Background on this error at: https://sqlalche.me/e/20/gkpj)

In [ ]:
def read_user(user_id: int, session=Depends(get_session)):
    statement = select(User).where(User.user_id == user_id)
    results = session.exec(statement).all()
    session.close()
    return results

read_user(1)

[User(username='bank', user_id=1, updated_dt=datetime.datetime(2025, 6, 23, 12, 53, 8, 671454), password='test', email='bank.bank@bank.com', created_dt=datetime.datetime(2025, 6, 23, 12, 53, 8, 671454))]

In [ ]:
def read_users(session=Depends(get_session)):
    statement = select(User)
    results = session.exec(statement).all()
    session.close()
    return results

read_users()

[User(username='bank', user_id=1, updated_dt=datetime.datetime(2025, 6, 23, 12, 53, 8, 671454), password='test', email='bank.bank@bank.com', created_dt=datetime.datetime(2025, 6, 23, 12, 53, 8, 671454)),
 User(username='bonk', user_id=2, updated_dt=datetime.datetime(2025, 6, 23, 12, 53, 8, 672455), password='test', email='bonk.bonk@bank.com', created_dt=datetime.datetime(2025, 6, 23, 12, 53, 8, 672455))]

## Update

In [5]:
from datetime import datetime, timezone

def update_user_email(user_id: int, new_email: str, session: Session = Depends(get_session)):
    user = session.get(User, user_id)
    if user:
        user.email = new_email
        user.updated_dt = datetime.now(timezone.utc)  # ✅ update timestamp
        session.add(user)
        session.commit()
        session.refresh(user)
        session.close()
    return user

update_user_email(user_id=1, new_email="bankbank@bank.com")

User(username='bank', user_id=1, updated_dt=datetime.datetime(2025, 6, 27, 17, 29, 14, 625214), email='bankbank@bank.com', password='test', created_dt=datetime.datetime(2025, 6, 23, 12, 53, 8, 671454))

In [6]:
def create_project(project: Project, session: Session = Depends(get_session)):
    session.add(project)
    session.commit()
    session.refresh(project)
    session.close()
    return project

project1 = Project(
    project_name="Project 1",
    owner_id=1
)

project2 = Project(
    project_name="Project 2",
    owner_id=1
)
create_project(project1)
create_project(project2)

Project(created_dt=datetime.datetime(2025, 6, 27, 17, 29, 15, 267956), project_id=5, owner_id=1, project_name='Project 2', updated_dt=datetime.datetime(2025, 6, 27, 17, 29, 15, 267956))

In [7]:
project = Project(
    project_name="Project 1",
    owner_id=2
)

create_project(project)

Project(created_dt=datetime.datetime(2025, 6, 27, 17, 29, 15, 880671), project_id=6, owner_id=2, project_name='Project 1', updated_dt=datetime.datetime(2025, 6, 27, 17, 29, 15, 880671))

In [8]:
def read_project(project_id:int, session:Session=Depends(get_session)):
    project = session.get(Project, project_id)
    session.close()
    return project

def read_projects(session:Session=Depends(get_session)):
    statement = select(Project)
    results = session.exec(statement).all()
    session.close()
    return results


read_project(project_id=1)

Project(created_dt=datetime.datetime(2025, 6, 23, 12, 53, 10, 585635), project_id=1, owner_id=1, project_name='Project 1', updated_dt=datetime.datetime(2025, 6, 23, 12, 53, 10, 585635))

In [9]:
read_project(project_id=2)

Project(created_dt=datetime.datetime(2025, 6, 23, 12, 53, 10, 585635), project_id=2, owner_id=1, project_name='Project 2', updated_dt=datetime.datetime(2025, 6, 23, 12, 53, 10, 585635))

In [10]:
read_projects()

[Project(created_dt=datetime.datetime(2025, 6, 23, 12, 53, 10, 585635), project_id=1, owner_id=1, project_name='Project 1', updated_dt=datetime.datetime(2025, 6, 23, 12, 53, 10, 585635)),
 Project(created_dt=datetime.datetime(2025, 6, 23, 12, 53, 10, 585635), project_id=2, owner_id=1, project_name='Project 2', updated_dt=datetime.datetime(2025, 6, 23, 12, 53, 10, 585635)),
 Project(created_dt=datetime.datetime(2025, 6, 23, 12, 53, 10, 807805), project_id=3, owner_id=2, project_name='Project 1', updated_dt=datetime.datetime(2025, 6, 23, 12, 53, 10, 807805)),
 Project(created_dt=datetime.datetime(2025, 6, 27, 17, 29, 15, 267956), project_id=4, owner_id=1, project_name='Project 1', updated_dt=datetime.datetime(2025, 6, 27, 17, 29, 15, 267956)),
 Project(created_dt=datetime.datetime(2025, 6, 27, 17, 29, 15, 267956), project_id=5, owner_id=1, project_name='Project 2', updated_dt=datetime.datetime(2025, 6, 27, 17, 29, 15, 267956)),
 Project(created_dt=datetime.datetime(2025, 6, 27, 17, 29, 1

In [11]:
def read_projects_by_user_id(user_id:int, session:Session=Depends(get_session)):
    user = session.get(User, user_id)
    projects = user.projects
    session.close()
    return projects

read_projects_by_user_id(user_id=1)

[Project(created_dt=datetime.datetime(2025, 6, 23, 12, 53, 10, 585635), project_id=1, owner_id=1, project_name='Project 1', updated_dt=datetime.datetime(2025, 6, 23, 12, 53, 10, 585635)),
 Project(created_dt=datetime.datetime(2025, 6, 23, 12, 53, 10, 585635), project_id=2, owner_id=1, project_name='Project 2', updated_dt=datetime.datetime(2025, 6, 23, 12, 53, 10, 585635)),
 Project(created_dt=datetime.datetime(2025, 6, 27, 17, 29, 15, 267956), project_id=4, owner_id=1, project_name='Project 1', updated_dt=datetime.datetime(2025, 6, 27, 17, 29, 15, 267956)),
 Project(created_dt=datetime.datetime(2025, 6, 27, 17, 29, 15, 267956), project_id=5, owner_id=1, project_name='Project 2', updated_dt=datetime.datetime(2025, 6, 27, 17, 29, 15, 267956))]

In [12]:
read_projects_by_user_id(user_id=2)

[Project(created_dt=datetime.datetime(2025, 6, 23, 12, 53, 10, 807805), project_id=3, owner_id=2, project_name='Project 1', updated_dt=datetime.datetime(2025, 6, 23, 12, 53, 10, 807805)),
 Project(created_dt=datetime.datetime(2025, 6, 27, 17, 29, 15, 880671), project_id=6, owner_id=2, project_name='Project 1', updated_dt=datetime.datetime(2025, 6, 27, 17, 29, 15, 880671))]

In [13]:
def read_user_by_project(project_id:int, session:Session=Depends(get_session)):
    project = session.get(Project, project_id)
    owner = project.owner
    session.close()
    return owner

read_user_by_project(project_id=1)

User(username='bank', user_id=1, updated_dt=datetime.datetime(2025, 6, 27, 17, 29, 14, 625214), email='bankbank@bank.com', password='test', created_dt=datetime.datetime(2025, 6, 23, 12, 53, 8, 671454))

In [14]:
read_user_by_project(project_id=3)

User(username='bonk', user_id=2, updated_dt=datetime.datetime(2025, 6, 23, 12, 53, 8, 672455), email='bonk.bonk@bank.com', password='test', created_dt=datetime.datetime(2025, 6, 23, 12, 53, 8, 672455))

In [15]:
from package.databases.models.longterm import LongTerm

def create_longterm(context:LongTerm, session:Session=Depends(get_session)):
    session.add(context)
    session.commit()
    session.refresh(context)
    session.close()
    return context

context1 = LongTerm(
    context="context 1",
    embedding=[0.5,0.5,0.5],
    meta={"source": "a", "sequence": 1}
)

context2 = LongTerm(
    context="context 2",
    embedding=[0.0, 0.0, 0.0],
    meta={"source": "b", "sequence": 2}
)

create_longterm(context1)
create_longterm(context2)

LongTerm(context='context 2', embedding=array([0., 0., 0.], dtype=float32), meta={'source': 'b', 'sequence': 2}, id=4)

In [16]:
def read_longterm(id:int, session:Session=Depends(get_session)):
    longterm = session.get(LongTerm, id)
    session.close()
    return longterm

read_longterm(id=1)

LongTerm(context='context 1', embedding=array([0.5, 0.5, 0.5], dtype=float32), meta={'source': 'a', 'sequence': 1}, id=1)

In [17]:
read_longterm(id=2)

LongTerm(context='context 2', embedding=array([0., 0., 0.], dtype=float32), meta={'source': 'b', 'sequence': 2}, id=2)

In [18]:
def read_similar_longterm(vector:list[float], limit:int=1, session:Session=Depends(get_session)):
    statement = select(LongTerm).order_by(LongTerm.embedding.l2_distance(vector)).limit(limit)
    results = session.exec(statement).all()
    session.close()
    return results

read_similar_longterm(vector=[0,0,0], limit=2)

[LongTerm(context='context 2', embedding=array([0., 0., 0.], dtype=float32), meta={'source': 'b', 'sequence': 2}, id=2),
 LongTerm(context='context 2', embedding=array([0., 0., 0.], dtype=float32), meta={'source': 'b', 'sequence': 2}, id=4)]

In [19]:
read_similar_longterm(vector=[0.5,0.5,0.5], limit=2)

[LongTerm(context='context 1', embedding=array([0.5, 0.5, 0.5], dtype=float32), meta={'source': 'a', 'sequence': 1}, id=1),
 LongTerm(context='context 1', embedding=array([0.5, 0.5, 0.5], dtype=float32), meta={'source': 'a', 'sequence': 1}, id=3)]

In [20]:
from sqlmodel import text
import json

def filter_longterm(params:dict, session:Session=Depends(get_session)):
    statement = select(LongTerm).where(
        text("meta @> :jsonb_filter")
    ).params(jsonb_filter=json.dumps(params))
    results = session.exec(statement).all()
    session.close()
    return results

filter_longterm(params={"source": "b"})

[LongTerm(context='context 2', embedding=array([0., 0., 0.], dtype=float32), meta={'source': 'b', 'sequence': 2}, id=2),
 LongTerm(context='context 2', embedding=array([0., 0., 0.], dtype=float32), meta={'source': 'b', 'sequence': 2}, id=4)]

In [21]:
from sqlmodel import text

def read_longterm(vector:list[float], limit:int=5, sources:list[str]=None, session:Session=Depends(get_session)):
    statement = select(LongTerm)
    if sources:
        statement=statement.where(text("meta ->> 'source' = ANY(:sources)"))
    statement = statement.order_by(LongTerm.embedding.l2_distance(vector)).limit(limit).params(sources=sources)
    results = session.exec(statement).all()
    session.close()
    return results

read_longterm(
    vector=[0,0,0],
    sources=["a"]
)

[LongTerm(context='context 1', embedding=array([0.5, 0.5, 0.5], dtype=float32), meta={'source': 'a', 'sequence': 1}, id=1),
 LongTerm(context='context 1', embedding=array([0.5, 0.5, 0.5], dtype=float32), meta={'source': 'a', 'sequence': 1}, id=3)]

In [22]:
read_longterm(
    vector=[0.5,0.5,0.5],
    sources=["a", "b"]
)

[LongTerm(context='context 1', embedding=array([0.5, 0.5, 0.5], dtype=float32), meta={'source': 'a', 'sequence': 1}, id=1),
 LongTerm(context='context 1', embedding=array([0.5, 0.5, 0.5], dtype=float32), meta={'source': 'a', 'sequence': 1}, id=3),
 LongTerm(context='context 2', embedding=array([0., 0., 0.], dtype=float32), meta={'source': 'b', 'sequence': 2}, id=2),
 LongTerm(context='context 2', embedding=array([0., 0., 0.], dtype=float32), meta={'source': 'b', 'sequence': 2}, id=4)]

## Delete

In [39]:
def delete_user(user_id: int, session: Session = Depends(get_session)):
    user = session.get(User, user_id)
    if user:
        session.delete(user)
        session.commit()
        session.close()
        return True
    return False

delete_user(user_id=1)


True

In [40]:
read_users()

[User(username='bonk', user_id=2, updated_dt=datetime.datetime(2025, 6, 22, 17, 5, 22, 935684), password='test', email='bonk.bonk@bank.com', created_dt=datetime.datetime(2025, 6, 22, 17, 5, 22, 935684))]

In [41]:
read_projects()

[Project(created_dt=datetime.datetime(2025, 6, 22, 17, 5, 26, 629407), project_id=3, owner_id=2, project_name='Project 1', updated_dt=datetime.datetime(2025, 6, 22, 17, 5, 26, 629407))]

# Drop the selected table

In [ ]:
from package.databases.drop import drop_table
# if there's relationship between two tables, it breaks
# drop_table(User)

✅ Dropped 'user' table.


# Drop all tables

In [1]:
from package.databases.drop import drop_all_tables

drop_all_tables()

✅ All tables dropped.
